# Prepare and Train Shadow Models for LiRA (Per-Example Calibration)

This notebook implements proper per-example LiRA with **32 shadow models**.

**Configuration:**
- **32 shadow models**: s ∈ {0, 1, ..., 31}
- **2000 candidate rows**: from target's test set (`./data/train/test.json`)
- **Per-example split**: Each test sample has exactly **16 IN** and **16 OUT** shadows (randomly assigned)

**Membership Matrix M[s,j]:**
- M[s,j] = 1 if shadow s was trained on example x_j, else 0
- For each example j: exactly 16 shadows have M[·,j] = 1 (IN)
- Which shadows are IN/OUT varies randomly per example

**Training Sets:**
```
For each shadow s:
  Train_s = WikiText_base ∪ {x_j | M[s,j] = 1}
```

**LiRA Attack:**
- For each test example x_j:
  - Get IN losses: from 16 shadows where M[s,j] = 1
  - Get OUT losses: from 16 shadows where M[s,j] = 0
  - Compute likelihood ratio: P(target_loss | IN) vs P(target_loss | OUT)

**Expected Performance:**
- With 32 shadows (16 IN/16 OUT): **15-30% TPR @ 1% FPR**
- Much better than 4 shadows (0.9% TPR) due to improved statistics
- Still limited by distribution mismatch (WikiText vs WikiText+train_finetune)

**Directories:**
- Dataset: `./data/shadow_datasets_lira/`
- Models: `./models/shadow_lira/`

**Training Time:** ~5-8 hours on RTX 5070 Ti

In [12]:
# Cell 1: Check GPU
import torch

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = torch.device("cuda")
else:
    print("⚠️  No GPU, training will be slow!")
    device = torch.device("cpu")

print(f"\nDevice: {device}")

PyTorch: 2.8.0+cu128
CUDA available: True
GPU: NVIDIA GeForce RTX 5070 Ti
Memory: 17.09 GB

Device: cuda


In [13]:
# Cell 2: Imports
import os
os.environ.setdefault("TRANSFORMERS_NO_TORCHVISION", "1")

import torch
import numpy as np
import json
import random
from pathlib import Path
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model
from tqdm.auto import tqdm
import gc

print("✓ Imports successful")

✓ Imports successful


In [14]:
# Cell 3: Configuration

# ========== LiRA CONFIGURATION ==========
NUM_SHADOW_MODELS = 32  # 32 shadow models (scaled up from 4)
IN_SHADOWS_PER_EXAMPLE = 16  # Each example: 16 IN, 16 OUT

# ========== DATASET CONFIGURATION ==========
TRAIN_PER_MODEL = 10_000  # WikiText base samples per shadow
TARGET_TEST_FILE = "./data/train/test.json"  # Target test set
MIN_TOKENS = 25  # Token filter
SHADOW_DATA_DIR = "./data/shadow_datasets_lira"

# ========== TRAINING CONFIGURATION ==========
MODEL_NAME = "gpt2"
BLOCK_SIZE = 512
EPOCHS = 3
BATCH_SIZE = 8
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 2e-4
LORA_R = 32
LORA_ALPHA = 64
LORA_DROPOUT = 0.05
SHADOW_MODEL_DIR = "./models/shadow_lira"

# ========== SEEDS ==========
SEED_START = 100
GLOBAL_SEED = 42

print("="*70)
print("PER-EXAMPLE LiRA CONFIGURATION (32 SHADOWS)")
print("="*70)
print(f"Shadows: {NUM_SHADOW_MODELS}")
print(f"Per-example: {IN_SHADOWS_PER_EXAMPLE} IN, {NUM_SHADOW_MODELS - IN_SHADOWS_PER_EXAMPLE} OUT")
print(f"\nWikiText base per shadow: {TRAIN_PER_MODEL:,}")
print(f"Target test set: {TARGET_TEST_FILE}")
print(f"Model: {MODEL_NAME} + LoRA (r={LORA_R})")
print(f"Training: {EPOCHS} epochs, lr={LEARNING_RATE}")
print(f"\nData: {SHADOW_DATA_DIR}/")
print(f"Models: {SHADOW_MODEL_DIR}/")
print(f"\n⚠️  Training time: ~{NUM_SHADOW_MODELS * 10}-{NUM_SHADOW_MODELS * 15} minutes (~5-8 hours)")
print("="*70)

PER-EXAMPLE LiRA CONFIGURATION (32 SHADOWS)
Shadows: 32
Per-example: 16 IN, 16 OUT

WikiText base per shadow: 10,000
Target test set: ./data/train/test.json
Model: gpt2 + LoRA (r=32)
Training: 3 epochs, lr=0.0002

Data: ./data/shadow_datasets_lira/
Models: ./models/shadow_lira/

⚠️  Training time: ~320-480 minutes (~5-8 hours)


In [15]:
# Cell 4: Helper Functions

def set_seed_all(seed: int):
    """Set all random seeds"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def basic_clean(ds: Dataset) -> Dataset:
    """Basic text cleaning"""
    ds = ds.filter(lambda ex: isinstance(ex.get("text", None), str) and len(ex["text"].strip()) > 0)
    def _strip(ex):
        return {"text": " ".join(ex["text"].split())}
    return ds.map(_strip, batched=False)

def filter_by_tokens(ds: Dataset, tok, min_tokens: int) -> Dataset:
    """Filter by token length"""
    def _len(batch):
        enc = tok(batch["text"], add_special_tokens=False)
        return {"_tok_len": [len(ids) for ids in enc["input_ids"]]}
    ds = ds.map(_len, batched=True)
    ds = ds.filter(lambda ex: ex["_tok_len"] >= min_tokens)
    return ds.remove_columns(["_tok_len"])

def cleanup_gpu():
    """Clean GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

print("✓ Helper functions defined")

✓ Helper functions defined


## Part 1: Dataset Generation

In [16]:
# Cell 6: Load WikiText-103

print("Loading WikiText-103...")
wiki_raw = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1", split="train")

print("Loading tokenizer...")
tok = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

print("Preprocessing...")
wiki = wiki_raw.remove_columns([c for c in wiki_raw.column_names if c != "text"])
wiki = basic_clean(wiki)
wiki = filter_by_tokens(wiki, tok, MIN_TOKENS)

print(f"✓ WikiText-103 ready: {len(wiki):,} samples")
print(f"  Need: {TRAIN_PER_MODEL * NUM_SHADOW_MODELS:,} for training")

Loading WikiText-103...
Loading tokenizer...
Preprocessing...
✓ WikiText-103 ready: 747,117 samples
  Need: 320,000 for training


In [17]:
# Cell 7: Load Target Test Set

print("="*70)
print("LOADING TARGET TEST SET")
print("="*70)

with open(TARGET_TEST_FILE, 'r') as f:
    test_texts = json.load(f)

TEST_SIZE = len(test_texts)

print(f"✓ Loaded {TEST_SIZE} test samples")
print(f"  From: {TARGET_TEST_FILE}")
print(f"  Preview: {test_texts[0][:80]}...")
print("="*70)

LOADING TARGET TEST SET
✓ Loaded 2000 test samples
  From: ./data/train/test.json
  Preview: Soundtrack to the Apocalypse 's ideal release date was in early November 2003 . ...


In [18]:
# Cell 8: Generate Membership Matrix M

print("="*70)
print("GENERATING MEMBERSHIP MATRIX M")
print("="*70)

set_seed_all(GLOBAL_SEED)

# M[j, s]: test sample j, shadow s
# For each test sample j, randomly choose IN_SHADOWS_PER_EXAMPLE shadows to be IN
keep_matrix = np.zeros((TEST_SIZE, NUM_SHADOW_MODELS), dtype=np.int32)

for j in range(TEST_SIZE):
    # Randomly select which shadows train on this example
    in_shadows = random.sample(range(NUM_SHADOW_MODELS), IN_SHADOWS_PER_EXAMPLE)
    for s in in_shadows:
        keep_matrix[j, s] = 1

print(f"✓ Matrix M: {keep_matrix.shape}")
print(f"  (test_samples={TEST_SIZE}, shadows={NUM_SHADOW_MODELS})")

print(f"\nPer-shadow statistics:")
for s in range(NUM_SHADOW_MODELS):
    num_in = keep_matrix[:, s].sum()
    print(f"  Shadow {s}: {num_in}/{TEST_SIZE} samples IN ({num_in/TEST_SIZE:.1%})")

print(f"\nPer-example statistics:")
in_counts = keep_matrix.sum(axis=1)
print(f"  Min: {in_counts.min()}, Max: {in_counts.max()}, Mean: {in_counts.mean():.2f}")
print(f"  Expected: {IN_SHADOWS_PER_EXAMPLE} IN per example")

# Verify
assert in_counts.min() == IN_SHADOWS_PER_EXAMPLE
assert in_counts.max() == IN_SHADOWS_PER_EXAMPLE
print(f"\n✓ Verified: All examples have exactly {IN_SHADOWS_PER_EXAMPLE} IN shadows")
print("="*70)

GENERATING MEMBERSHIP MATRIX M
✓ Matrix M: (2000, 32)
  (test_samples=2000, shadows=32)

Per-shadow statistics:
  Shadow 0: 1032/2000 samples IN (51.6%)
  Shadow 1: 960/2000 samples IN (48.0%)
  Shadow 2: 1019/2000 samples IN (50.9%)
  Shadow 3: 1010/2000 samples IN (50.5%)
  Shadow 4: 1046/2000 samples IN (52.3%)
  Shadow 5: 993/2000 samples IN (49.6%)
  Shadow 6: 974/2000 samples IN (48.7%)
  Shadow 7: 993/2000 samples IN (49.6%)
  Shadow 8: 1002/2000 samples IN (50.1%)
  Shadow 9: 985/2000 samples IN (49.2%)
  Shadow 10: 1024/2000 samples IN (51.2%)
  Shadow 11: 970/2000 samples IN (48.5%)
  Shadow 12: 1026/2000 samples IN (51.3%)
  Shadow 13: 1018/2000 samples IN (50.9%)
  Shadow 14: 992/2000 samples IN (49.6%)
  Shadow 15: 1002/2000 samples IN (50.1%)
  Shadow 16: 940/2000 samples IN (47.0%)
  Shadow 17: 1016/2000 samples IN (50.8%)
  Shadow 18: 1022/2000 samples IN (51.1%)
  Shadow 19: 979/2000 samples IN (48.9%)
  Shadow 20: 987/2000 samples IN (49.4%)
  Shadow 21: 996/2000 samp

In [19]:
# Cell 9: Generate Shadow Datasets

print("="*70)
print("GENERATING SHADOW DATASETS")
print("="*70)

os.makedirs(SHADOW_DATA_DIR, exist_ok=True)
used_indices = set()

for shadow_id in tqdm(range(NUM_SHADOW_MODELS), desc="Datasets"):
    seed = SEED_START + shadow_id
    set_seed_all(seed)
    
    shadow_dir = Path(SHADOW_DATA_DIR) / f"shadow_{shadow_id}"
    train_file = shadow_dir / "train_finetune.json"
    keep_file = shadow_dir / "keep.npy"
    
    if train_file.exists():
        print(f"\nShadow {shadow_id}: exists, skipping")
        continue
    
    # Sample WikiText base
    available = [i for i in range(len(wiki)) if i not in used_indices]
    if len(available) < TRAIN_PER_MODEL:
        available = list(range(len(wiki)))
        used_indices.clear()
    
    random.Random(seed).shuffle(available)
    selected = sorted(available[:TRAIN_PER_MODEL])
    used_indices.update(selected)
    
    base_texts = [wiki[i]["text"] for i in selected]
    
    # Get test samples for this shadow based on M
    keep_array = keep_matrix[:, shadow_id]
    in_indices = np.where(keep_array == 1)[0]
    test_samples_in = [test_texts[i] for i in in_indices]
    
    # Combine
    all_texts = base_texts + test_samples_in
    random.Random(seed + 999).shuffle(all_texts)
    
    print(f"\nShadow {shadow_id}:")
    print(f"  WikiText: {len(base_texts):,}")
    print(f"  Test IN: {len(test_samples_in):,}")
    print(f"  Total: {len(all_texts):,}")
    
    # Save
    shadow_dir.mkdir(parents=True, exist_ok=True)
    
    with open(train_file, "w") as f:
        json.dump([{"text": t} for t in all_texts], f, ensure_ascii=False, indent=2)
    
    np.save(keep_file, keep_array)

# Save shared files
test_file = Path(SHADOW_DATA_DIR) / "test.json"
with open(test_file, "w") as f:
    json.dump(test_texts, f, ensure_ascii=False, indent=2)

keep_matrix_path = Path(SHADOW_DATA_DIR) / "keep_matrix.npy"
np.save(keep_matrix_path, keep_matrix)

print(f"\n{'='*70}")
print(f"✅ {NUM_SHADOW_MODELS} shadow datasets generated")
print(f"📁 {SHADOW_DATA_DIR}/")
print(f"✓ test.json: {test_file}")
print(f"✓ keep_matrix.npy: {keep_matrix_path}")
print("="*70)

GENERATING SHADOW DATASETS


Datasets:   0%|          | 0/32 [00:00<?, ?it/s]


Shadow 0: exists, skipping

Shadow 1: exists, skipping

Shadow 2: exists, skipping

Shadow 3: exists, skipping


Datasets:  16%|█▌        | 5/32 [00:01<00:09,  2.96it/s]


Shadow 4:
  WikiText: 10,000
  Test IN: 1,046
  Total: 11,046


...


Shadow 29:
  WikiText: 10,000
  Test IN: 964
  Total: 10,964


Datasets:  97%|█████████▋| 31/32 [00:23<00:00,  1.35it/s]


Shadow 30:
  WikiText: 10,000
  Test IN: 1,003
  Total: 11,003


Datasets: 100%|██████████| 32/32 [00:23<00:00,  1.34it/s]


Shadow 31:
  WikiText: 10,000
  Test IN: 1,003
  Total: 11,003

✅ 32 shadow datasets generated
📁 ./data/shadow_datasets_lira/
✓ test.json: data/shadow_datasets_lira/test.json
✓ keep_matrix.npy: data/shadow_datasets_lira/keep_matrix.npy


## Part 2: Train Shadow Models

In [20]:
# Cell 11: Training Function

def train_shadow_model(shadow_id, data_dir, output_dir):
    """Train a single shadow model with LoRA"""
    print(f"\n{'='*70}")
    print(f"TRAINING SHADOW {shadow_id}")
    print(f"{'='*70}")
    
    seed = SEED_START + shadow_id
    set_seed_all(seed)
    
    # Load data
    with open(Path(data_dir) / "train_finetune.json") as f:
        train_items = json.load(f)
    
    ds = Dataset.from_list(train_items)
    train_raw = ds.filter(lambda ex: ex.get("text") and len(ex["text"].strip()) > 0)
    
    # Check keep array
    keep_file = Path(data_dir) / "keep.npy"
    if keep_file.exists():
        keep_arr = np.load(keep_file)
        print(f"✓ Keep array: {keep_arr.sum()} test samples IN")
    
    print(f"Training samples: {len(train_raw):,}")
    
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    def tokenize_fn(ex):
        texts = [(t if t and t.strip() else tokenizer.eos_token) for t in ex["text"]]
        return tokenizer(
            texts,
            padding="max_length",
            truncation=True,
            max_length=BLOCK_SIZE,
            return_attention_mask=True,
        )
    
    train_tok = train_raw.map(tokenize_fn, batched=True, remove_columns=train_raw.column_names)
    collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
    
    # Model
    print(f"Loading {MODEL_NAME}...")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
    model.resize_token_embeddings(len(tokenizer))
    model.gradient_checkpointing_enable()
    
    # LoRA
    print(f"Applying LoRA (r={LORA_R})...")
    lora_config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        target_modules=["c_attn", "c_fc", "c_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    # Training args
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        warmup_ratio=0.03,
        logging_steps=50,
        save_strategy="no",
        seed=seed,
        bf16=torch.cuda.is_available() and torch.cuda.is_bf16_supported(),
        fp16=torch.cuda.is_available() and not torch.cuda.is_bf16_supported(),
        report_to="none",
        gradient_checkpointing=False,
        optim="adamw_torch",
        dataloader_num_workers=2,
    )
    
    # Train
    print(f"\n🚀 Training on {device}...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tok,
        tokenizer=tokenizer,
        data_collator=collator,
    )
    
    trainer.train()
    
    # Save
    print(f"Saving to {output_dir}")
    os.makedirs(output_dir, exist_ok=True)
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    # Copy keep array
    if keep_file.exists():
        import shutil
        shutil.copy(keep_file, Path(output_dir) / "keep.npy")
        print("✓ Copied keep.npy")
    
    print(f"✅ Shadow {shadow_id} complete")
    print("="*70)
    
    # Cleanup
    del model, trainer
    cleanup_gpu()

print("✓ Training function defined")

✓ Training function defined


In [21]:
# Cell 12: Train All Shadows

print("="*70)
print("TRAINING SHADOW MODELS")
print("="*70)
print(f"⚠️  ~{NUM_SHADOW_MODELS * 10}-{NUM_SHADOW_MODELS * 15} minutes on GPU")
print("="*70)

os.makedirs(SHADOW_MODEL_DIR, exist_ok=True)

trained = []
skipped = []
failed = []

for shadow_id in range(NUM_SHADOW_MODELS):
    data_dir = Path(SHADOW_DATA_DIR) / f"shadow_{shadow_id}"
    output_dir = Path(SHADOW_MODEL_DIR) / f"shadow_{shadow_id}"
    
    # Skip if exists
    if output_dir.exists() and (output_dir / "adapter_model.safetensors").exists():
        print(f"\n⏭️  Shadow {shadow_id}: exists, skipping")
        skipped.append(shadow_id)
        continue
    
    if not data_dir.exists():
        print(f"\n❌ Shadow {shadow_id}: dataset not found")
        failed.append(shadow_id)
        continue
    
    try:
        train_shadow_model(shadow_id, data_dir, output_dir)
        trained.append(shadow_id)
    except Exception as e:
        print(f"\n❌ Shadow {shadow_id}: {e}")
        failed.append(shadow_id)
        cleanup_gpu()

print(f"\n{'='*70}")
print("TRAINING COMPLETE")
print(f"{'='*70}")
print(f"✅ Trained: {len(trained)} {trained}")
print(f"⏭️  Skipped: {len(skipped)} {skipped}")
print(f"❌ Failed: {len(failed)} {failed}")
print("="*70)

if len(trained) + len(skipped) == NUM_SHADOW_MODELS:
    print(f"\n🎉 All {NUM_SHADOW_MODELS} shadows ready for LiRA!")
else:
    print(f"\n⚠️  Only {len(trained) + len(skipped)}/{NUM_SHADOW_MODELS} ready")

TRAINING SHADOW MODELS
⚠️  ~320-480 minutes on GPU

⏭️  Shadow 0: exists, skipping

⏭️  Shadow 1: exists, skipping

⏭️  Shadow 2: exists, skipping

⏭️  Shadow 3: exists, skipping

TRAINING SHADOW 4


Filter: 100%|██████████| 11046/11046 [00:00<00:00, 319936.21 examples/s]


✓ Keep array: 1046 test samples IN
Training samples: 11,046


Map: 100%|██████████| 11046/11046 [00:02<00:00, 4906.83 examples/s]


Loading gpt2...
Applying LoRA (r=32)...
trainable params: 4,718,592 || all params: 129,158,400 || trainable%: 3.6533

🚀 Training on cuda...


...

Loading gpt2...
Applying LoRA (r=32)...
trainable params: 4,718,592 || all params: 129,158,400 || trainable%: 3.6533

🚀 Training on cuda...


/tmp/ipykernel_123273/3817591224.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if 

Step,Training Loss
50,4.047500
100,3.806400
150,3.604100
200,3.623300
250,3.583800
300,3.555500
350,3.543800
400,3.507100
450,3.526100
500,3.528100


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Saving to models/shadow_lira/shadow_31
✓ Copied keep.npy
✅ Shadow 31 complete

TRAINING COMPLETE
✅ Trained: 28 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
⏭️  Skipped: 4 [0, 1, 2, 3]
❌ Failed: 0 []

🎉 All 32 shadows ready for LiRA!


## Part 3: Verification

In [22]:
# Cell 14: Verify Setup

print("="*70)
print("VERIFICATION")
print("="*70)

test_file = Path(SHADOW_DATA_DIR) / "test.json"
keep_matrix_file = Path(SHADOW_DATA_DIR) / "keep_matrix.npy"

print(f"\n📁 Shared files:")
if test_file.exists():
    with open(test_file) as f:
        print(f"  ✓ test.json: {len(json.load(f))} samples")
else:
    print(f"  ❌ test.json: MISSING")

if keep_matrix_file.exists():
    M = np.load(keep_matrix_file)
    print(f"  ✓ keep_matrix.npy: {M.shape}")
else:
    print(f"  ❌ keep_matrix.npy: MISSING")

print(f"\n📁 Datasets:")
for i in range(NUM_SHADOW_MODELS):
    d = Path(SHADOW_DATA_DIR) / f"shadow_{i}"
    if (d / "train_finetune.json").exists():
        with open(d / "train_finetune.json") as f:
            cnt = len(json.load(f))
        keep_arr = np.load(d / "keep.npy")
        print(f"  ✓ shadow_{i}: {cnt:,} train, {keep_arr.sum()} IN")
    else:
        print(f"  ❌ shadow_{i}: MISSING")

print(f"\n🤖 Models:")
for i in range(NUM_SHADOW_MODELS):
    m = Path(SHADOW_MODEL_DIR) / f"shadow_{i}"
    if (m / "adapter_model.safetensors").exists():
        size_mb = (m / "adapter_model.safetensors").stat().st_size / 1024 / 1024
        has_keep = (m / "keep.npy").exists()
        print(f"  ✓ shadow_{i}: {size_mb:.1f} MB, keep: {has_keep}")
    else:
        print(f"  ❌ shadow_{i}: NOT TRAINED")

print("="*70)

VERIFICATION

📁 Shared files:
  ✓ test.json: 2000 samples
  ✓ keep_matrix.npy: (2000, 32)

📁 Datasets:
  ✓ shadow_0: 10,988 train, 988 IN
  ✓ shadow_1: 10,984 train, 984 IN
  ✓ shadow_2: 11,010 train, 1010 IN
  ✓ shadow_3: 11,018 train, 1018 IN
  ✓ shadow_4: 11,046 train, 1046 IN
  ✓ shadow_5: 10,993 train, 993 IN
  ✓ shadow_6: 10,974 train, 974 IN
  ✓ shadow_7: 10,993 train, 993 IN
  ✓ shadow_8: 11,002 train, 1002 IN
  ✓ shadow_9: 10,985 train, 985 IN
  ✓ shadow_10: 11,024 train, 1024 IN
  ✓ shadow_11: 10,970 train, 970 IN
  ✓ shadow_12: 11,026 train, 1026 IN
  ✓ shadow_13: 11,018 train, 1018 IN
  ✓ shadow_14: 10,992 train, 992 IN
  ✓ shadow_15: 11,002 train, 1002 IN
  ✓ shadow_16: 10,940 train, 940 IN
  ✓ shadow_17: 11,016 train, 1016 IN
  ✓ shadow_18: 11,022 train, 1022 IN
  ✓ shadow_19: 10,979 train, 979 IN
  ✓ shadow_20: 10,987 train, 987 IN
  ✓ shadow_21: 10,996 train, 996 IN
  ✓ shadow_22: 10,977 train, 977 IN
  ✓ shadow_23: 11,011 train, 1011 IN
  ✓ shadow_24: 11,029 train, 102

## How to Use for LiRA Attack

**Per-Example LiRA Algorithm with 32 Shadows:**

```python
# 1. Load membership matrix M
M = np.load("./data/shadow_datasets_lira/keep_matrix.npy")  # shape: (2000, 32)

# 2. Compute losses for all models on all test samples
NUM_SHADOW_MODELS = 32
shadow_losses = np.zeros((NUM_SHADOW_MODELS, TEST_SIZE))
for s in range(NUM_SHADOW_MODELS):
    for j in range(TEST_SIZE):
        shadow_losses[s, j] = compute_loss(shadow_model_s, test_sample_j)

target_losses = np.array([compute_loss(target_model, test_sample_j) for j in range(TEST_SIZE)])

# 3. For each test example j, compute LiRA score
lira_scores = np.zeros(TEST_SIZE)

for j in range(TEST_SIZE):
    # Get IN and OUT shadows for this specific example
    in_mask = (M[j, :] == 1)   # 16 shadows that trained on x_j
    out_mask = (M[j, :] == 0)  # 16 shadows that didn't train on x_j
    
    # Extract losses
    in_losses = shadow_losses[in_mask, j]    # 16 values (much better statistics!)
    out_losses = shadow_losses[out_mask, j]  # 16 values
    
    # Compute statistics for THIS example
    in_mean = in_losses.mean()
    in_std = in_losses.std()
    out_mean = out_losses.mean()
    out_std = out_losses.std()
    
    # LiRA score (normalized)
    combined_std = np.sqrt(in_std**2 + out_std**2)
    lira_scores[j] = (out_mean - in_mean) / (combined_std + 1e-9)

# 4. Evaluate
from sklearn.metrics import roc_auc_score, roc_curve
labels = load_labels()  # 1 = member, 0 = non-member

auc = roc_auc_score(labels, lira_scores)
fpr, tpr, _ = roc_curve(labels, lira_scores)
tpr_at_1_fpr = tpr[np.where(fpr <= 0.01)[0][-1]]

print(f"AUC: {auc:.4f}")
print(f"TPR @ 1% FPR: {tpr_at_1_fpr:.4f}")
```

**Key Improvements with 32 Shadows:**
- Each example has 16 IN losses + 16 OUT losses (vs 2+2 before)
- Much more reliable mean and std estimates
- Expected: **15-30% TPR @ 1% FPR** (vs 0.9% with 4 shadows)
- Still limited by distribution mismatch, but significantly better!

**Next Step:**
Update `MIA_LiRA.ipynb` Cell 2: Change `NUM_SHADOW_MODELS = 4` to `NUM_SHADOW_MODELS = 32`